## Codigo del EDA que finalmente no se uso

In [ ]:
# === 5. Tratamiento de valores nulos ===
# Para numéricas: imputar con la mediana
#for col in num_vars:
#    if ech[col].isnull().any():
#        ech[col] = ech[col].fillna(ech[col].median())

# Para categóricas: imputar con una categoría especial
#for col in cat_vars:
#    if ech[col].isnull().any():
#        ech[col] = ech[col].fillna('No informado')

In [ ]:
# === 6. Codificación de variables categóricas ===
# Convertir binarias codificadas como 1/2 o Sí/No a 0/1
#def codificar_binarias(df):
#    for col in df.columns:
#        if df[col].nunique() == 2:
#            vals = sorted(df[col].dropna().unique())
#            if set(vals) <= set([0,1,2]):
#                df[col] = df[col].map({vals[0]: 0, vals[1]: 1})
#    return df

#ech = codificar_binarias(ech)

# One-hot encoding para variables categóricas no binarias
#ech_encoded = pd.get_dummies(ech, columns=cat_vars, drop_first=True)

In [ ]:
# ====== PASO 2: Preprocesamiento (ORIGINAL)====== 

# Codificación de ordinales – si los valores están bien ordenados
#ordinal_encoder = OrdinalEncoder()

# Escalador para continuas
#scaler = StandardScaler()

# Codificador para nominales
#onehot = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')

# ColumnTransformer para aplicar todo junto
#preprocessor = ColumnTransformer(transformers=[
#    ('nom', onehot, nominales),
#    ('ord', ordinal_encoder, ordinales),
#    ('scale', scaler, numericas_continuas)
#], remainder='passthrough')  # deja discretas y dicotómicas sin tocar

In [ ]:
# ====== PASO 3: Separar X e y ======

#X = ech[dicotomicas + nominales + ordinales + numericas_discretas + numericas_continuas]

# Eliminar variables con fuga de información
#vars_a_eliminar = ['YDA', 'YDA_SVL']  # ajustar según auditoría
#X = X.drop(columns=[col for col in vars_a_eliminar if col in X.columns])

#y = ech['log_YDA']

In [ ]:
# ====== PASO 5: Pipeline final ======

#pipeline = Pipeline(steps=[
#    ('preprocesamiento', preprocessor)
#])

# Fit-transform solo para ver el shape de salida
#X_train_proc = pipeline.fit_transform(X_train)
#X_test_proc = pipeline.transform(X_test)
#print("Shape del dataset procesado:", X_train_proc.shape)

In [ ]:
# 3) Filtrar listas de variables para que no incluyan columnas eliminadas
#dicotomicas = [c for c in dicotomicas if c not in vars_a_eliminar]
#nominales = [c for c in nominales if c not in vars_a_eliminar]
#ordinales = [c for c in ordinales if c not in vars_a_eliminar]
#numericas_discretas = [c for c in numericas_discretas if c not in vars_a_eliminar]
#numericas_continuas = [c for c in numericas_continuas if c not in vars_a_eliminar]

In [ ]:
import os

# Crear carpeta "data_processed" dentro del proyecto si no existe
output_dir = "data_processed"
os.makedirs(output_dir, exist_ok=True)

# Guardar X e y en CSV
X.to_csv(os.path.join(output_dir, "X_clean.csv"), index=False)
y.to_csv(os.path.join(output_dir, "y_clean.csv"), index=False)

print("✅ Archivos guardados en carpeta 'data_processed':")
print(f"- {os.path.join(output_dir, 'X_clean.csv')}")
print(f"- {os.path.join(output_dir, 'y_clean.csv')}")

Funciones de limpieza que haciamos luego del train_test_split:

In [ ]:
# ============================================
# 2️⃣ Funciones de limpieza
# ============================================
def eliminar_varianza_cero(df):
    from sklearn.feature_selection import VarianceThreshold
    selector = VarianceThreshold(threshold=0.0)
    selector.fit(df)
    cols_quitar = df.columns[~selector.get_support()].tolist()
    df_filtrado = df.drop(columns=cols_quitar)
    return df_filtrado, cols_quitar

def eliminar_correlacion_perfecta(df):
    corr_matrix = df.corr()
    cols_quitar = set()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if corr_matrix.iloc[i, j] == 1:
                colname = corr_matrix.columns[i]
                cols_quitar.add(colname)
    df_filtrado = df.drop(columns=list(cols_quitar))
    return df_filtrado, list(cols_quitar)

def eliminar_correlacion_alta(df, umbral=0.95):
    corr_matrix = df.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    cols_quitar = [column for column in upper.columns if any(upper[column] > umbral)]
    df_filtrado = df.drop(columns=cols_quitar)
    return df_filtrado, cols_quitar

In [ ]:
# ============================================
# 3️⃣ Aplicar limpieza a TRAIN y TEST
# ============================================
# Varianza cero
X_train_proc, cols_var0 = eliminar_varianza_cero(X_train_proc)
X_test_proc = X_test_proc.drop(columns=cols_var0, errors='ignore')

# Correlación perfecta
X_train_proc, cols_corr1 = eliminar_correlacion_perfecta(X_train_proc)
X_test_proc = X_test_proc.drop(columns=cols_corr1, errors='ignore')

# Correlación alta
X_train_proc, cols_corr_high = eliminar_correlacion_alta(X_train_proc, umbral=0.95)
X_test_proc = X_test_proc.drop(columns=cols_corr_high, errors='ignore')

print(f"📉 Shape final tras limpieza: {X_train_proc.shape}")

In [ ]:
# ============================================
# 4️⃣ Guardar columnas eliminadas para trazabilidad
# ============================================
df_eliminadas = pd.DataFrame({
    "varianza_cero": pd.Series(cols_var0),
    "corr_perfecta": pd.Series(cols_corr1),
    "corr_alta": pd.Series(cols_corr_high)
})
df_eliminadas.to_excel("columnas_eliminadas.xlsx", index=False)
print("📂 Archivo 'columnas_eliminadas.xlsx' guardado.")

In [ ]:
import pandas as pd

# Cargar datasets limpios
X = pd.read_csv("data_processed/X_clean.csv")
y = pd.read_csv("data_processed/y_clean.csv").squeeze()  # .squeeze() para que sea Serie y no DataFrame

print("Shapes cargados:")
print("X:", X.shape)
print("y:", y.shape)

In [ ]:
# Importo librerias
#from sklearn.linear_model import LinearRegression, Ridge, Lasso
#from sklearn.tree import DecisionTreeRegressor
#from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
#from sklearn.model_selection import cross_val_score
#from sklearn.metrics import make_scorer, mean_squared_error
#import numpy as np
#import pandas as pd

In [ ]:
# Importar XGBoost y LightGBM si están disponibles
#try:
#    from xgboost import XGBRegressor
#except ImportError:
#    XGBRegressor = None

#try:
#    from lightgbm import LGBMRegressor
#except ImportError:
#    LGBMRegressor = None

In [ ]:
# Definir modelos a evaluar
#modelos = {
#    "LinearRegression": LinearRegression(),
#    "Ridge": Ridge(),
#    "Lasso": Lasso(),
#    "DecisionTree": DecisionTreeRegressor(),
#    "RandomForest": RandomForestRegressor(n_jobs=-1),
#    "GradientBoosting": GradientBoostingRegressor()
#}

In [ ]:
# Agregar XGBoost si está disponible
#if XGBRegressor:
#    modelos["XGBoost"] = XGBRegressor(n_jobs=-1)

# Agregar LightGBM si está disponible
#if LGBMRegressor:
#    modelos["LightGBM"] = LGBMRegressor(n_jobs=-1)

In [ ]:
# Métrica: MSE negativo (lo convierte a positivo más abajo)
#scoring = make_scorer(mean_squared_error, greater_is_better=False)

In [ ]:
# Ejecuta validación cruzada y muestra resultados
#resultados = {}
#for nombre, modelo in modelos.items():
#    scores = cross_val_score(modelo, X_train_proc, y_train, cv=5, scoring=scoring)
#    mse_promedio = -np.mean(scores)
#    print(f"{nombre}: MSE promedio = {mse_promedio:,.2f}")
#    resultados[nombre] = mse_promedio

In [ ]:
# Mostrar resultados ordenados
#df_resultados = pd.DataFrame(resultados).T.sort_values(by='RMSE promedio')
#print("\nComparación de modelos por RMSE (menor es mejor):")
#display(df_resultados)

Primera funcion para evaluar modelo (la comentamos para no correrla).

In [ ]:
# Utilidad para evaluar en ambas escalas
#def evaluar_modelo(modelo, Xtr, ytr, Xte, yte, nombre="Modelo"):
    # Entrenar
#    modelo.fit(Xtr, ytr)
    # Predecir
#    y_pred = modelo.predict(Xte)

    # Métricas en escala log (si y es log_YDA)
#    mae = mean_absolute_error(yte, y_pred)
#    mse = mean_squared_error(yte, y_pred)
#    rmse = np.sqrt(mse)
#    r2 = r2_score(yte, y_pred)

#    print(f"\n=== {nombre} | Métricas en escala log (log_YDA) ===")
#    print(f"MAE : {mae:,.4f}")
#    print(f"MSE : {mse:,.4f}")
#    print(f"RMSE: {rmse:,.4f}")
#    print(f"R²  : {r2:,.4f}")

    # Métricas en escala original (deslogarizando)
#    yte_orig = np.expm1(yte)
#    y_pred_orig = np.expm1(y_pred)

#    mae_o = mean_absolute_error(yte_orig, y_pred_orig)
#    mse_o = mean_squared_error(yte_orig, y_pred_orig)
#    rmse_o = np.sqrt(mse_o)
#    r2_o = r2_score(yte_orig, y_pred_orig)

#    print(f"\n=== {nombre} | Métricas en escala original (pesos) ===")
#    print(f"MAE : {mae_o:,.2f}")
#    print(f"MSE : {mse_o:,.2f}")
#    print(f"RMSE: {rmse_o:,.2f}")
#    print(f"R²  : {r2_o:,.4f}")

#    return {
#        "mae_log": mae, "mse_log": mse, "rmse_log": rmse, "r2_log": r2,
#        "mae": mae_o, "mse": mse_o, "rmse": rmse_o, "r2": r2_o,
#        "y_pred": y_pred, "y_pred_orig": y_pred_orig
#    }

Funcion para evaluar modelos con metricas solo en test.

In [ ]:
 #print("📊 Calculando métricas en escala log...")
    #r2 = r2_score(y_test, y_pred_test)
    #mae = mean_absolute_error(y_test, y_pred_test)
    #rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

    # 🔁 Revertimos a escala de pesos, evitando overflow en np.exp
    #y_test_clip = np.clip(y_test, 0, 30)
    #y_pred_clip = np.clip(y_pred_test, 0, 30)

    #y_test_original = np.exp(y_test_clip)
    #y_pred_original = np.exp(y_pred_clip)

    #print(f"Máximo y_pred_original (clipped): {np.max(y_pred_original)}")

    #print("📊 Calculando métricas en pesos...")
    #mae_pesos = mean_absolute_error(y_test_original, y_pred_original)
    #rmse_pesos = np.sqrt(mean_squared_error(y_test_original, y_pred_original))

    #print("✅ Métricas finales:")
    #print(f"Modelo: {nombre}")
    #print(f"R2: {r2:.4f} | MAE_log: {mae:.2f} | RMSE_log: {rmse:.2f}")
    #print(f"MAE_pesos: {mae_pesos:.2f} | RMSE_pesos: {rmse_pesos:.2f}")

    #return {
    #    "modelo": nombre,
    #    "R2": r2,
    #    "MAE_log": mae,
    #    "RMSE_log": rmse,
    #    "MAE_pesos": mae_pesos,
    #    "RMSE_pesos": rmse_pesos
    #}

In [ ]:
# 1) Split en crudo (sin preprocesar aún)

# X = ech[dicotomicas + nominales + ordinales + numericas_discretas + numericas_continuas]
# y = ech['log_YDA']   # ya definido previamente

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
# 2) Ajustar SOLO con TRAIN y transformar TRAIN y TEST:
# El preprocessor es el ColumnTransformer definido antes.

#preprocessor.fit(X_train)

#X_train_proc = preprocessor.transform(X_train)
#X_test_proc  = preprocessor.transform(X_test)

#print("Shapes tras preprocesamiento:")
#print("X_train_proc:", X_train_proc.shape)
#print("X_test_proc :", X_test_proc.shape)

In [ ]:
modelo_lr = LinearRegression()

try:
    modelo_lr.fit(X_train_proc, y_train)
    print("✅ Fit correcto")
except Exception as e:
    print("❌ Error durante el fit:", e)

try:
    y_pred = modelo_lr.predict(X_test_proc)
    print("✅ Predict correcto")
except Exception as e:
    print("❌ Error durante el predict:", e)

In [ ]:
# 4) Fit-transform SOLO con train y transform test
X_train_proc = preprocessor.fit_transform(X_train)
X_test_proc  = preprocessor.transform(X_test)

print("Shapes tras preprocesamiento:")
print("X_train_proc:", X_train_proc.shape)
print("X_test_proc :", X_test_proc.shape)

Hacemos un VIF (Variance Inflation Factor) de las variables procesadas para detectar multicolinealidad.

In [ ]:
# =========================================
# 1) Calcular VIF en datos procesados
# =========================================

from statsmodels.stats.outliers_influence import variance_inflation_factor

# X_train_proc es un array numpy, así que lo pasamos a DataFrame
# Usamos las columnas generadas por el preprocessor
feature_names = preprocessor.get_feature_names_out()
X_proc_df = pd.DataFrame(X_proc, columns=feature_names)

# Calculamos VIF
vif_data = pd.DataFrame()
vif_data["feature"] = X_proc_df.columns
vif_data["VIF"] = [
    variance_inflation_factor(X_proc_df.values, i) for i in range(X_proc_df.shape[1])
]

# Ordenamos por VIF descendente
vif_data = vif_data.sort_values(by="VIF", ascending=False)
print("\n📊 Top 20 features con mayor VIF:")
print(vif_data.head(20))

📊 Top 20 features con mayor VIF:
              feature  VIF
1442  nom__f72_2_8413  inf
1409  nom__f72_2_7210  inf
1423  nom__f72_2_7830  inf
1422  nom__f72_2_7820  inf
1421  nom__f72_2_7810  inf
1420  nom__f72_2_7730  inf
1419  nom__f72_2_7729  inf
1418  nom__f72_2_7721  inf
1417  nom__f72_2_7710  inf
1416  nom__f72_2_7500  inf
1415  nom__f72_2_7490  inf
1414  nom__f72_2_7420  inf
1413  nom__f72_2_7410  inf
1412  nom__f72_2_7320  inf
1411  nom__f72_2_7310  inf
1410  nom__f72_2_7220  inf
1408  nom__f72_2_7120  inf
1590      nom__f305_2  inf
1407  nom__f72_2_7110  inf
1406  nom__f72_2_7020  inf

Esos resultados son una señal clarísima de que tienes multicolinealidad perfecta en muchas variables, y además concentrada en un bloque muy específico: las columnas nom__f72_2_XXXX y alguna como nom__f305_2.

Que el VIF sea inf significa que alguna de esas variables es combinación lineal exacta de otras, lo cual es típico cuando:

Hay One-Hot Encoding con categorías que nunca se presentan en train o tienen representación redundante.

Se incluyen todas las dummies de una variable categórica (no se hace drop='first').

Hay columnas que son copias exactas o que sumadas a otras siempre dan lo mismo (dummy trap).

💡 Conclusiones prácticas:

Tu OneHotEncoder probablemente está generando columnas completamente redundantes en algunas categorías (las de f72_2 parecen ser un caso de codificación con muchísimas categorías).

Esto explica por qué Linear Regression explotaba** → la matriz X'X es singular o casi singular.

Ridge puede estabilizar algo, pero si el VIF es inf es porque tienes redundancia total; lo ideal sería reducir esas columnas.

Dado que ya usas drop='first' en el OneHotEncoder, podría estar pasando que algunas categorías están 100% correlacionadas con otras variables (por ejemplo, dummies para ubicaciones que coinciden con otra variable geográfica).

Modelo de Ridge Regression original que no usaba la funcion evaluar_modelo:

In [ ]:
# =========================================
# 2) Entrenar Ridge Regression
# =========================================

from sklearn.linear_model import Ridge

modelo_ridge = Ridge(alpha=10)  # alpha controla regularización; probar 1, 10, 100
modelo_ridge.fit(X_train, y_train)

y_pred = modelo_ridge.predict(X_test_proc)

# Evaluación
mae_log = mean_absolute_error(y_test, y_pred)
mse_log = mean_squared_error(y_test, y_pred)
rmse_log = np.sqrt(mse_log)
r2_log = r2_score(y_test, y_pred)

# Invertimos escala log para métricas en pesos
y_pred_original = np.exp(y_pred)
y_test_original = np.exp(y_test)

mae_pesos = mean_absolute_error(y_test_original, y_pred_original)
rmse_pesos = np.sqrt(mean_squared_error(y_test_original, y_pred_original))

print("\n✅ Métricas Ridge Regression:")
print(f"R2: {r2_log:.4f} | MAE_log: {mae_log:.4f} | RMSE_log: {rmse_log:.4f}")
print(f"MAE_pesos: {mae_pesos:.2f} | RMSE_pesos: {rmse_pesos:.2f}")

Auditoría de features (Feature Importance)

Esto te permite ver cuáles son las variables más influyentes según el modelo. Si aparece una variable que contiene información directa del ingreso (por ejemplo, otro tipo de ingreso o una suma de ingresos del hogar), entonces es probable que haya fuga de información.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Obtener nombres de las columnas procesadas (después del pipeline)
# Esto asume que hiciste fit en el pipeline con X_train
feature_names = preprocessor.get_feature_names_out()
importancias = modelo_rf.feature_importances_

# Crear DataFrame con los resultados
importancias_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importancias
}).sort_values(by='importance', ascending=False)

# Mostrar las 20 más importantes
print(importancias_df.head(20))

# Graficar
plt.figure(figsize=(10,6))
plt.barh(importancias_df.head(20)['feature'][::-1], importancias_df.head(20)['importance'][::-1])
plt.xlabel("Importancia")
plt.title("Top 20 variables más importantes - Random Forest")
plt.tight_layout()
plt.show()

Observamos:
- La variable num__YDA aporta el 99.8% de la importancia del modelo.
- El resto de las variables tienen importancias casi nulas (valores del orden de 0.001 o incluso 0.000002).
- Entre las más importantes figuran también HT11, YDA_SVL y YSVL, todas ellas potencialmente relacionadas con ingreso

Interpretación:
- Esto es una señal clara de fuga de información.
- YDA es tu variable objetivo (aunque aquí en su versión logarítmica), y si aparece como predictor, entonces el modelo está simplemente "viendo" la respuesta, lo cual hace inútiles las métricas obtenidas.
- Variables como YDA_SVL o YSVL también suenan sospechosamente similares, por lo que es probable que sean funciones directas o indirectas del ingreso.

Conclusión de esta parte:
- Tu modelo está dominado por variables que contienen directamente (o de forma colineal) la variable que intentás predecir. 
- Esto invalida los resultados. Se debe excluir YDA y toda variable derivada del conjunto de predictores.



Random Forest con Validación cruzada (Cross-Validation)

Esto te dice si el rendimiento es estable entre distintas particiones de los datos. Si hay sobreajuste, el resultado promedio será mucho más bajo que el obtenido en tu partición test anterior.

In [ ]:
from sklearn.model_selection import cross_val_score

# Pipeline que combina preprocesamiento y modelo
modelo_rf_cv = Pipeline(steps=[
    ('preprocesamiento', preprocessor),
    ('modelo', RandomForestRegressor(random_state=42))
])

# Ejecutar CV con el pipeline
scores = cross_val_score(modelo_rf_cv, X, y, cv=5, scoring='r2')

print("R² por fold:", scores)
print(f"Media R²: {scores.mean():.4f}")
print(f"Desviación estándar: {scores.std():.4f}")

Observamos:

Nivel de rendimiento:
- Media R² ≈ 0.8576 significa que, en promedio, el modelo explica alrededor del 85,8% de la variabilidad del ingreso en los datos de validación.
- Esto es menor al R² ≈ 0.94 que obtuviste en el train/test split único, lo que es normal porque la validación cruzada da una estimación más realista y menos optimista.

Estabilidad del modelo:
- Desviación estándar: 0.0127 → muy baja, lo que indica que el rendimiento es consistente entre folds y que el modelo es estable ante cambios en el conjunto de entrenamiento.
- Un desvío bajo sugiere que el modelo no depende demasiado de un subconjunto particular de los datos.

Posible sobreajuste:
- La diferencia entre el R² alto en train/test único y el R² de CV sugiere que el modelo podría estar sobreajustando levemente.
- Esto se puede trabajar ajustando hiperparámetros (max_depth, min_samples_split, etc.) o reduciendo dimensionalidad.

Conclusión:
- El modelo es fuerte y estable, pero el gap entre test único y CV sugiere que todavía hay margen para mejorar generalización.
- Ya está en un rango de rendimiento alto para este tipo de problema, así que es un buen candidato para afinar con GridSearch o RandomizedSearch.

Random Forest con GridSearchCV explota la memoria.

In [ ]:
# 5) Random Forest con GridSearchCV (Modelo 5)

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Búsqueda en una grilla compacta pero informativa
param_grid_rf = {
    "n_estimators": [300, 500, 800],
    "max_depth": [None, 12, 20],
    "min_samples_split": [2, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", "log2"]
}

rf_base = RandomForestRegressor(random_state=42, n_jobs=-1)

# Usamos RMSE negativo como scoring (minimizar RMSE)
grid_rf = GridSearchCV(
    estimator=rf_base,
    param_grid=param_grid_rf,
    scoring="neg_root_mean_squared_error",
    cv=3,
    n_jobs=-1,
    verbose=1
)

# Ajuste con TRAIN ya preprocesado
grid_rf.fit(X_train_proc, y_train)

print("\n=== GridSearchCV RandomForest ===")
print("Mejores hiperparámetros:", grid_rf.best_params_)
print(f"Mejor score CV (RMSE): {-grid_rf.best_score_:,.4f}")

# Evaluación final en TEST con el mejor estimador
mejor_rf = grid_rf.best_estimator_
res_rf_cv = evaluar_modelo(mejor_rf, X_train_proc, y_train, X_test_proc, y_test, nombre="Random Forest (GridSearchCV)")

Recordamos:
- Los árboles/boosting suelen rendir mejor que los lineales en este tipo de datos tabulares con alta no linealidad e interacciones.
- Si el tiempo es un factor, podés reducir la grilla del RF (menos combinaciones) o usar RandomizedSearchCV.
- Si tu y es log_YDA, las métricas “originales” que imprime el bloque (deslogarizando) son las más interpretables para presentación.

In [ ]:
# RandomizedSearchCV XGBoost

#import numpy as np
#import pandas as pd
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, mean_squared_error

# Scoring de RMSE (negativo para que lo maximice)
scoring_rmse = "neg_root_mean_squared_error"

try:
    from xgboost import XGBRegressor
    from scipy.stats import randint, uniform, loguniform

    xgb_base = XGBRegressor(
        random_state=42,
        n_jobs=-1,
        tree_method="hist"   # rápido y estable
    )

    dist_xgb = {
        "n_estimators": randint(400, 1500),
        "max_depth": randint(3, 12),
        "learning_rate": loguniform(1e-3, 3e-1),
        "subsample": uniform(0.6, 0.4),         # [0.6, 1.0]
        "colsample_bytree": uniform(0.6, 0.4),  # [0.6, 1.0]
        "reg_lambda": loguniform(1e-3, 10),
        "reg_alpha": loguniform(1e-4, 1)
    }

    rnd_xgb = RandomizedSearchCV(
        estimator=xgb_base,
        param_distributions=dist_xgb,
        n_iter=40,              # podés subir/bajar según tiempo
        scoring=scoring_rmse,
        cv=3,
        n_jobs=-1,
        random_state=42,
        verbose=1
    )

    rnd_xgb.fit(X_train_proc, y_train)
    print("\n=== RandomizedSearchCV XGBoost ===")
    print("Mejores hiperparámetros:", rnd_xgb.best_params_)
    print(f"Mejor score CV (RMSE): {-rnd_xgb.best_score_:,.4f}")

    best_xgb = rnd_xgb.best_estimator_
    res_xgb = evaluar_modelo(best_xgb, X_train_proc, y_train, X_test_proc, y_test, nombre="XGBoost (RandomizedSearchCV)")

except ImportError:
    best_xgb = None
    print("\n[AVISO] xgboost no está instalado. Ejecuta: pip install xgboost")

In [ ]:
# RandomizedSearchCV LightGBM

try:
    from lightgbm import LGBMRegressor
    from scipy.stats import randint, uniform, loguniform

    lgb_base = LGBMRegressor(
        random_state=42,
        n_jobs=-1
    )

    dist_lgb = {
        "n_estimators": randint(400, 1500),
        "learning_rate": loguniform(1e-3, 3e-1),
        "num_leaves": randint(31, 255),
        "max_depth": [-1, 6, 8, 10, 12],          # categórico
        "subsample": uniform(0.6, 0.4),
        "colsample_bytree": uniform(0.6, 0.4),
        "reg_lambda": loguniform(1e-3, 10),
        "min_child_samples": randint(5, 100)
    }

    rnd_lgb = RandomizedSearchCV(
        estimator=lgb_base,
        param_distributions=dist_lgb,
        n_iter=40,
        scoring=scoring_rmse,
        cv=3,
        n_jobs=-1,
        random_state=42,
        verbose=1
    )

    rnd_lgb.fit(X_train_proc, y_train)
    print("\n=== RandomizedSearchCV LightGBM ===")
    print("Mejores hiperparámetros:", rnd_lgb.best_params_)
    print(f"Mejor score CV (RMSE): {-rnd_lgb.best_score_:,.4f}")

    best_lgb = rnd_lgb.best_estimator_
    res_lgb = evaluar_modelo(best_lgb, X_train_proc, y_train, X_test_proc, y_test, nombre="LightGBM (RandomizedSearchCV)")

except ImportError:
    best_lgb = None
    print("\n[AVISO] lightgbm no está instalado. Ejecuta: pip install lightgbm")

In [ ]:
# Permutation Importance (sobre el mejor modelo)

from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt

# 1) Obtener nombres de features del preprocessor ya fiteado
try:
    feature_names = preprocessor.get_feature_names_out(X.columns)
except Exception:
    # fallback: índices si no hay nombres
    feature_names = np.array([f"f{i}" for i in range(X_train_proc.shape[1])])

# 2) Elegir el mejor modelo entre XGB y LGB (por RMSE en test)
candidatos = []
if 'res_xgb' in locals():
    candidatos.append(("XGBoost", best_xgb, res_xgb['rmse']))
if 'res_lgb' in locals():
    candidatos.append(("LightGBM", best_lgb, res_lgb['rmse']))

if candidatos:
    ganador_nombre, ganador_modelo, _ = sorted(candidatos, key=lambda t: t[2])[0]
    print(f"\nModelo ganador para interpretación: {ganador_nombre}")
else:
    # Si no hay XGB/LGB disponibles, podés elegir otro (ej: mejor RF)
    ganador_nombre, ganador_modelo = "Modelo_no_disponible", None

# 3) Permutation importance (si hay modelo)
if ganador_modelo is not None:
    # Para acelerar, usar una muestra del test (opcional)
    np.random.seed(42)
    idx_sample = np.random.choice(X_test_proc.shape[0], size=min(3000, X_test_proc.shape[0]), replace=False)
    X_te_pi = X_test_proc[idx_sample]
    y_te_pi = y_test.iloc[idx_sample] if hasattr(y_test, "iloc") else y_test[idx_sample]

    print("\nCalculando Permutation Importance (esto puede tardar un poco)...")
    pi = permutation_importance(
        estimator=ganador_modelo,
        X=X_te_pi,
        y=y_te_pi,
        n_repeats=5,
        scoring="neg_root_mean_squared_error",
        random_state=42,
        n_jobs=-1
    )

    importancias = pd.DataFrame({
        "feature": feature_names,
        "importance_mean": pi.importances_mean,
        "importance_std": pi.importances_std
    }).sort_values("importance_mean", ascending=False)

    top = importancias.head(20)
    print("\nTop 20 features por Permutation Importance:")
    display(top)

    # Plot
    plt.figure(figsize=(8, 10))
    plt.barh(top["feature"][::-1], top["importance_mean"][::-1])
    plt.title(f"Permutation Importance - Top 20 ({ganador_nombre})")
    plt.xlabel("Mean decrease in score (|neg RMSE|)")
    plt.tight_layout()
    plt.show()
else:
    print("\n[AVISO] No hay modelo ganador para calcular permutation importance.")

# SHAP para el modelo ganador

try:
    import shap

    if ganador_modelo is not None:
        # Crear DataFrame con nombres de columnas (mejora los gráficos)
        X_te_df = pd.DataFrame(X_test_proc, columns=feature_names)

        # Muestra para acelerar el cómputo
        np.random.seed(42)
        samp = min(2000, X_te_df.shape[0])
        X_te_sample = X_te_df.sample(samp, random_state=42)

        # Explainer para modelos de árboles (XGB/LGB/RF)
        explainer = shap.TreeExplainer(ganador_modelo)
        shap_values = explainer.shap_values(X_te_sample)

        # Resumen global
        shap.summary_plot(shap_values, X_te_sample, show=True, max_display=25)
        # (Opcional) resumen por bar:
        shap.summary_plot(shap_values, X_te_sample, plot_type="bar", show=True, max_display=25)

        # Nota: los SHAP están en la misma escala que el target (log_YDA).
    else:
        print("\n[AVISO] No hay modelo ganador para calcular SHAP.")

except ImportError:
    print("\n[AVISO] shap no está instalado. Ejecuta: pip install shap")

Consideramos:
- Nombres de features: preprocessor.get_feature_names_out(X.columns) te devuelve los nombres ya expandidos por el OneHot; así los gráficos y tablas quedan legibles.
- Tiempo de cómputo: tanto RandomizedSearchCV como SHAP pueden tardar; por eso limito n_iter y tomo muestras de test para importancia/SHAP.
- Interpretación: SHAP y permutation importance reportan contribuciones en escala de y. Como vos usás log_YDA, las contribuciones están en log-ingreso (lo señalás en el informe).

Tabla comparativa de modelos

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Función auxiliar para evaluar en log y en escala original (pesos)
def evaluar_modelo_full(nombre, modelo, X_tr_proc, X_te_proc, y_tr, y_te):
    # En log
    y_pred_train = modelo.predict(X_tr_proc)
    y_pred_test  = modelo.predict(X_te_proc)

    mae_log  = mean_absolute_error(y_te, y_pred_test)
    rmse_log = np.sqrt(mean_squared_error(y_te, y_pred_test))
    r2_log   = r2_score(y_te, y_pred_test)

    # A escala original
    y_te_pesos    = np.exp(y_te)
    y_pred_pesos  = np.exp(y_pred_test)

    mae_pesos  = mean_absolute_error(y_te_pesos, y_pred_pesos)
    rmse_pesos = np.sqrt(mean_squared_error(y_te_pesos, y_pred_pesos))
    r2_pesos   = r2_score(y_te_pesos, y_pred_pesos)

    return {
        "Modelo": nombre,
        "MAE_log": mae_log,
        "RMSE_log": rmse_log,
        "R2_log": r2_log,
        "MAE_pesos": mae_pesos,
        "RMSE_pesos": rmse_pesos,
        "R2_pesos": r2_pesos
    }

# Lista de modelos ya entrenados y sus nombres
modelos_resultados = []

# Modelo 1: Regresión lineal
modelos_resultados.append(evaluar_modelo_full("LinearRegression", modelo_lr, X_train_proc, X_test_proc, y_train, y_test))

# Modelo 2: Random Forest básico
modelos_resultados.append(evaluar_modelo_full("RandomForest", modelo_rf, X_train_proc, X_test_proc, y_train, y_test))

# Modelo 3: XGBoost optimizado
modelos_resultados.append(evaluar_modelo_full("XGBoost", best_xgb, X_train_proc, X_test_proc, y_train, y_test))

# Modelo 4: LightGBM optimizado
modelos_resultados.append(evaluar_modelo_full("LightGBM", best_lgb, X_train_proc, X_test_proc, y_train, y_test))

# Modelo 5: Random Forest con GridSearchCV
modelos_resultados.append(evaluar_modelo_full("RandomForest (tuned)", best_rf, X_train_proc, X_test_proc, y_train, y_test))

# Crear tabla resumen
tabla_resultados = pd.DataFrame(modelos_resultados)

# Ordenar por RMSE_log ascendente (mejor modelo arriba)
tabla_resultados = tabla_resultados.sort_values(by="RMSE_log")

# Mostrar con 3 decimales
pd.set_option("display.float_format", lambda x: f"{x:,.3f}")
display(tabla_resultados)

Cross Validation original con los tres modelos haciendo fit transform:

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd
import time

# =====================
# 1) Preprocesar con float32 para reducir memoria
# =====================
X_train_proc = preprocessor.fit_transform(X_train).astype(np.float32)
X_test_proc  = preprocessor.transform(X_test).astype(np.float32)

# =====================
# 2) Función para evaluación rápida
# =====================
def evaluar_modelo_simple(modelo, X_train, y_train, X_test, y_test, nombre="Modelo"):
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)

    mae_log = mean_absolute_error(y_test, y_pred)
    rmse_log = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    # Revertimos log a pesos
    y_test_original = np.exp(y_test)
    y_pred_original = np.exp(y_pred)
    mae_pesos = mean_absolute_error(y_test_original, y_pred_original)
    rmse_pesos = np.sqrt(mean_squared_error(y_test_original, y_pred_original))

    return {
        "Modelo": nombre,
        "R2": r2,
        "MAE_log": mae_log,
        "RMSE_log": rmse_log,
        "MAE_pesos": mae_pesos,
        "RMSE_pesos": rmse_pesos
    }

# =====================
# 3) RandomizedSearchCV para cada modelo
# =====================

# --- Random Forest ---
param_rf = {
    "n_estimators": [200, 400, 600],
    "max_depth": [10, 15, 20, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}
rf = RandomForestRegressor(random_state=42, n_jobs=-1)
search_rf = RandomizedSearchCV(rf, param_rf, n_iter=10, cv=3, scoring='r2',
                               verbose=2, random_state=42, n_jobs=-1)
t0 = time.time()
search_rf.fit(X_train_proc, y_train)
print("Random Forest mejores params:", search_rf.best_params_, "Tiempo:", round(time.time()-t0, 2), "s")

# --- XGBoost ---
param_xgb = {
    "n_estimators": [200, 400, 600],
    "max_depth": [3, 5, 7, 10],
    "learning_rate": [0.01, 0.05, 0.1],
    "subsample": [0.7, 0.8, 1.0],
    "colsample_bytree": [0.7, 0.8, 1.0]
}
xgb = XGBRegressor(objective='reg:squarederror', random_state=42, n_jobs=-1)
search_xgb = RandomizedSearchCV(xgb, param_xgb, n_iter=10, cv=3, scoring='r2',
                                verbose=2, random_state=42, n_jobs=-1)
t0 = time.time()
search_xgb.fit(X_train_proc, y_train)
print("XGBoost mejores params:", search_xgb.best_params_, "Tiempo:", round(time.time()-t0, 2), "s")

# --- LightGBM ---
param_lgb = {
    "n_estimators": [200, 400, 600],
    "max_depth": [-1, 10, 15, 20],
    "learning_rate": [0.01, 0.05, 0.1],
    "subsample": [0.7, 0.8, 1.0],
    "colsample_bytree": [0.7, 0.8, 1.0]
}
lgb = LGBMRegressor(objective='regression', random_state=42, n_jobs=-1)
search_lgb = RandomizedSearchCV(lgb, param_lgb, n_iter=10, cv=3, scoring='r2',
                                verbose=2, random_state=42, n_jobs=-1)
t0 = time.time()
search_lgb.fit(X_train_proc, y_train)
print("LightGBM mejores params:", search_lgb.best_params_, "Tiempo:", round(time.time()-t0, 2), "s")

# =====================
# 4) Evaluar y comparar
# =====================
resultados = []
resultados.append(evaluar_modelo_simple(search_rf.best_estimator_, X_train_proc, y_train, X_test_proc, y_test, "Random Forest"))
resultados.append(evaluar_modelo_simple(search_xgb.best_estimator_, X_train_proc, y_train, X_test_proc, y_test, "XGBoost"))
resultados.append(evaluar_modelo_simple(search_lgb.best_estimator_, X_train_proc, y_train, X_test_proc, y_test, "LightGBM"))

df_resultados = pd.DataFrame(resultados)
print("\n📊 Comparativa de modelos:")
print(df_resultados)